In [ ]:
from vespa.application import Vespa
from sentence_transformers import SentenceTransformer
from pprint import pprint

vespa_conn = Vespa(url="http://localhost", port=8080)

model_name = 'intfloat/multilingual-e5-base'
model = SentenceTransformer(model_name)

In [ ]:
def run_multiple_ranking(keyword: str):

    results = []
    for rank in ('bm25', 'semantic', 'fusion'):
        result = vespa_conn.query(
            yql='select * from doc where ({targetHits:100}nearestNeighbor(embedding, q)) or userQuery()',
            query=keyword,
            body={
                "input.query(q)": model.encode(keyword, normalize_embeddings=True).tolist()
            },
            ranking=rank,
            language='ja',
            tracelevel=2
        )
        print(rank)
        pprint(result.hits[0])
        print()
        print()
        results.append(result)

    return results

In [ ]:
keyword = 'AMDのパフォーマンス'
results = run_multiple_ranking(keyword)

In [ ]:
results[-1].get_json()